In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Read images from Kaggle input directory
img1 = plt.imread('/kaggle/input/myimages/uttower_right.jpg') 
img2 = plt.imread('/kaggle/input/myimages/uttower_left.jpg')

# Convert images to grayscale
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Detect SIFT features and compute descriptors
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Use Flann-based Matcher to find keypoint matches
flann = cv2.FlannBasedMatcher()
matches = flann.knnMatch(des1, des2, 2)

# Select good matches using the ratio test
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)

# Draw and display matched features
matched_img = cv2.drawMatches(img1, kp1, img2, kp2, good, None)

plt.imshow(matched_img)
plt.show()

In [ ]:
def compute_homography(pts1, pts2):
    # Set up A matrix
    A = []

    # Construct the A matrix for each corresponding pair of points
    for i in range(pts1.shape[0]):
        x, y = pts1[i, 0], pts1[i, 1] 
        u, v = pts2[i, 0], pts2[i, 1]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    A = np.array(A)
    
    # Solve Ax = 0 using Singular Value Decomposition (SVD)
    U, S, Vt = np.linalg.svd(A)
    
    # Homography matrix H is the last column of Vt transposed
    H = Vt[-1].reshape(3, 3)  
    H = H / H[2, 2]  # Normalize by dividing by the last element
    return H


In [ ]:
# Get points from good matches        
pts1 = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1,2)
pts2 = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1,2)
print(pts1.shape)
# Compute homography
normal_homography = compute_homography(pts1, pts2)
print(normal_homography)

In [ ]:
# RANSAC parameters
ransac_iterations = 1000
ransac_threshold = 5.0

# RANSAC loop
best_homography = None
max_inliers = 0

# Iterate through RANSAC iterations
for iteration in range(ransac_iterations):
    # Randomly select 4 matches for computing homography
    random_matches = np.random.choice(good, 4, replace=False)
    src_pts = np.float32([kp1[m.queryIdx].pt for m in random_matches]).reshape(-1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in random_matches]).reshape(-1, 2)

    # Compute homography using the randomly selected matches
    H = compute_homography(src_pts, dst_pts)

    # Use the current homography (H) for transforming the points
    transformed_pts = cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), H)

    # Calculate Sum of Square Differences (SSD) as the inlier measure
    ssd = np.sum((pts2 - transformed_pts.squeeze())**2, axis=-1)
    inliers = np.sum(ssd < ransac_threshold**2)

    # Update if the current homography has more inliers
    if inliers > max_inliers:
        max_inliers = inliers
        best_homography = H

# Recompute best homography using all inliers
ssd_all = np.sum((pts2 - cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), best_homography).squeeze())**2, axis=-1)
all_inliers = ssd_all < ransac_threshold**2

# Use np.where to find the indices where all_inliers is True
inlier_indices = np.where(all_inliers)[0]

# Extract the corresponding inlier points from pts1 and pts2
all_src_pts = np.float32(pts1[inlier_indices])
all_dst_pts = np.float32(pts2[inlier_indices])

# Recompute the best homography using all inliers
best_homography = compute_homography(all_src_pts, all_dst_pts)

# Compare the quality of homographies
normal_inliers = np.sum(np.abs(pts2 - cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), normal_homography)) < ransac_threshold)
ransac_inliers = np.sum(np.abs(pts2 - cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), best_homography)) < ransac_threshold)

# Calculate Sum of Square Differences (SSD) as the error measure
ssd_normal = np.sum((pts2 - cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), normal_homography).squeeze())**2, axis=-1)
ssd_ransac = np.sum((pts2 - cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), best_homography).squeeze())**2, axis=-1)

# Calculate the mean SSD as the reprojection error
normal_reprojection_error = np.mean(ssd_normal)
ransac_reprojection_error = np.mean(ssd_ransac)

# Print the results
print("Number of Inliers:")
print("Normal Homography:", normal_inliers)
print("RANSAC-Based Homography:", ransac_inliers)

print("\nReprojection Error:")
print("Normal Homography:", normal_reprojection_error)
print("RANSAC-Based Homography:", ransac_reprojection_error)